# Проверяем файлы в кластере и устанавливаем библиотеки

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
import pyspark
app_name = "myApp" # Ваше имя
spark_ui_port = 4040 # Порт для spark ui

spark = (
    pyspark.sql.SparkSession.builder
        .appName(app_name) # имя приложения ? Нужно для отслеживания таски выполнения
        .config("spark.executor.cores", "4")        
        .config("spark.executor.memory", "4g") # Executor просее. Ориенир для потребления помаяти. 
        .config("spark.executor.instances", "6")    
        .config("spark.default.parallelism", "48")
        .config("spark.driver.memory", "4g") # Main процесс
        .config("spark.ui.port", spark_ui_port)
        .getOrCreate()
)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter

In [3]:
df = spark.read.parquet('s3a://mlopsshakhova/output_fin.parquet')

In [4]:
import pandas as pd

print('Data overview')
df.printSchema()
print('Columns overview')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Data overview
root
 |-- tranaction_id: integer (nullable = true)
 |-- tx_datetime: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: float (nullable = true)
 |-- tx_time_seconds: integer (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)

Columns overview


,Column Name,Data type
0,tranaction_id,int
1,tx_datetime,timestamp
2,customer_id,int
3,terminal_id,int
4,tx_amount,float
5,tx_time_seconds,int
6,tx_time_days,int
7,tx_fraud,int
8,tx_fraud_scenario,int


In [5]:
df.show()

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|          148|2019-08-22 10:01:24|         95|          0|    16.69|          36084|           0|       0|                0|
|          463|2019-08-22 16:54:03|        307|        683|    68.07|          60843|           0|       0|                0|
|          471|2019-08-22 08:34:07|        312|         63|   123.38|          30847|           0|       0|                0|
|          496|2019-08-22 06:58:00|        335|        289|   166.28|          25080|           0|       0|                0|
|          833|2019-08-22 20:24:06|        531|        189|    28.71|          73446|           0|       0|           

In [6]:
df.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
tranaction_id,1879122447,9.399026627123606E8,5.426495235886582E8,0,1879791584
customer_id,1879122447,500428.5022185784,288559.10721771803,0,999999
terminal_id,1879122447,501.18719838271403,317.54928231300045,0,999
tx_amount,1879122447,54.232534740877654,41.28705509290461,0.01,16539.04
tx_time_seconds,1879122447,5.184082983006759E7,2.9929863070764977E7,0,103679999
tx_time_days,1879122447,599.5118438053547,346.4097189561111,0,1199
tx_fraud,1879122447,0.05924712366548618,0.23608664094461965,0,1
tx_fraud_scenario,1879122447,0.11934065784697638,0.47755787299001473,0,3


Итак, видно, что строк стало меньше, чем в начальном файле. Также были удалены строки с некорректными id

In [7]:
spark.stop()